# Austin Shelter Wrangle Notes <a name='top'></a>

This notebook contains notes and code to develop the `final_adoption_report` for the <a href='https://github.com/stephenfitzsimon/pet_adoption_project'>Austin Shelter Pet Outcomes</a> project.

- two tables are downloaded
- duplicate `animal_id` rows are dropped
- columns dropped: `animal_id_i`
- datetime columns are transformed to month-year
- animal_type, breed, and color were consistent, dropped the intake column
- name column had nulls that appeared as mismatched.  replaced with string
- drop 15 NaN outcome_types
    - outcome_subtype NaN are replaced with no subtype
    - SCRP is Stray Cat Release Program
- drop nulls for sex_outcome, age_outcome, sex_intake
- converted age_at_intake and age_at_outcome into days

### Contents

1. <a href='#download'>Getting data from the internet</a>
2. <a href='#joining'>Joining the table data</a>
3. <a href='#datetime'>Handling the datetime columns </a>
4. <a href='#integrity'>Checking data integrity of select columns </a>
5. <a href='#scripts'>Testing the scripts</a>

In [1]:
import os
import requests
import pandas as pd
from sodapy import Socrata

## Getting the data from the internet <a name='download'></a>

Write a function that downloads the data from the internet.  Use the <a href='https://dev.socrata.com/'>Socrata Open Data API.</a>

In [2]:
def download_data():
    """
    Returns the pet outcome and pet intake dataframes from the SODA
    """
    client = Socrata("data.austintexas.gov", None)
    results_outcome = client.get("9t4d-g238", limit=200_000)
    results_intake = client.get("wter-evkm", limit=200_000)

    # Convert to pandas DataFrame
    df_outcome = pd.DataFrame.from_records(results_outcome)
    df_intake = pd.DataFrame.from_records(results_intake)
    return df_outcome, df_intake

#df_o, df_i = download_data()

In [3]:
df_i

,animal_id,name,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color
0,A860516,A860516,2022-06-29T13:43:00.000,2022-06-29T13:43:00.000,Highway 290 And Decker Lane in Austin (TX),Stray,Injured,Cat,Unknown,3 months,Domestic Shorthair Mix,Orange Tabby
1,A860515,A860515,2022-06-29T13:34:00.000,2022-06-29T13:34:00.000,8306 Turnberry Lane in Austin (TX),Stray,Normal,Cat,Unknown,1 month,Domestic Shorthair Mix,Brown Tabby
2,A799377,*Comet,2022-06-29T13:07:00.000,2022-06-29T13:07:00.000,6301 Harold Court in Austin (TX),Stray,Normal,Dog,Neutered Male,3 years,Labrador Retriever Mix,Fawn/Black
3,A860512,A860512,2022-06-29T13:07:00.000,2022-06-29T13:07:00.000,6301 Harold Court in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Pit Bull,Tricolor
4,A860509,A860509,2022-06-29T13:05:00.000,2022-06-29T13:05:00.000,12138 North Lamar in Austin (TX),Stray,Normal,Cat,Intact Female,2 weeks,Domestic Shorthair Mix,White/Black
...,...,...,...,...,...,...,...,...,...,...,...,...
141298,A664233,Stevie,2013-10-01T08:53:00.000,2013-10-01T08:53:00.000,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White
141299,A664236,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
141300,A664237,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
141301,A664235,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White


Use the function to check for a `.csv` file. Allow for the user to force a url query

In [4]:
def get_pet_data(query_url = False):
    file_o = 'pet_outcomes.csv'
    file_i = 'pet_intake.csv'
    if os.path.isfile(file_o) and os.path.isfile(file_i) and not query_url:
        #return dataframe from file
        print('Returning saved csv files.')
        df_o = pd.read_csv(file_o).drop(columns = ['Unnamed: 0'])
        df_i = pd.read_csv(file_i).drop(columns = ['Unnamed: 0'])
        return df_o, df_i
    else:
        print('Getting data from url...')
        df_o, df_i = download_data()
        print('Saving to .csv files...')
        df_o.to_csv(file_o)
        df_i.to_csv(file_i)
        print('Returned dataframes.')
        return df_o, df_i

#df_o, df_i = get_pet_data(query_url=True)

Getting data from url...
Saving to .csv files...
Returned dataframes.


In [5]:
df_i

,animal_id,name,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color
0,A860516,A860516,2022-06-29T13:43:00.000,2022-06-29T13:43:00.000,Highway 290 And Decker Lane in Austin (TX),Stray,Injured,Cat,Unknown,3 months,Domestic Shorthair Mix,Orange Tabby
1,A860515,A860515,2022-06-29T13:34:00.000,2022-06-29T13:34:00.000,8306 Turnberry Lane in Austin (TX),Stray,Normal,Cat,Unknown,1 month,Domestic Shorthair Mix,Brown Tabby
2,A799377,*Comet,2022-06-29T13:07:00.000,2022-06-29T13:07:00.000,6301 Harold Court in Austin (TX),Stray,Normal,Dog,Neutered Male,3 years,Labrador Retriever Mix,Fawn/Black
3,A860512,A860512,2022-06-29T13:07:00.000,2022-06-29T13:07:00.000,6301 Harold Court in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Pit Bull,Tricolor
4,A860509,A860509,2022-06-29T13:05:00.000,2022-06-29T13:05:00.000,12138 North Lamar in Austin (TX),Stray,Normal,Cat,Intact Female,2 weeks,Domestic Shorthair Mix,White/Black
...,...,...,...,...,...,...,...,...,...,...,...,...
141298,A664233,Stevie,2013-10-01T08:53:00.000,2013-10-01T08:53:00.000,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White
141299,A664236,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
141300,A664237,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
141301,A664235,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White


In [6]:
df_o

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A859339,*Bodhi,2022-06-29T13:37:00.000,2022-06-29T13:37:00.000,2021-02-14T00:00:00.000,Adoption,Dog,Spayed Female,1 year,German Shepherd,Black/Brown,NaN
1,A741260,Kayla,2022-06-29T12:36:00.000,2022-06-29T12:36:00.000,2014-12-30T00:00:00.000,Return to Owner,Dog,Spayed Female,7 years,Pit Bull Mix,Black/White,NaN
2,A860179,Alloy,2022-06-29T12:36:00.000,2022-06-29T12:36:00.000,2018-06-23T00:00:00.000,Transfer,Dog,Neutered Male,4 years,German Shepherd Mix,Black/Brown,Partner
3,A860475,NaN,2022-06-29T12:13:00.000,2022-06-29T12:13:00.000,2022-05-26T00:00:00.000,Transfer,Cat,Intact Female,4 weeks,Domestic Shorthair,Black,Partner
4,A860471,A860471,2022-06-29T11:46:00.000,2022-06-29T11:46:00.000,2022-05-16T00:00:00.000,Euthanasia,Cat,Intact Male,NULL,Domestic Shorthair,Brown Tabby/White,Suffering
...,...,...,...,...,...,...,...,...,...,...,...,...
141165,A664223,Moby,2013-10-01T11:03:00.000,2013-10-01T11:03:00.000,2009-09-30T00:00:00.000,Return to Owner,Dog,Neutered Male,4 years,Bulldog Mix,White,NaN
141166,A664236,NaN,2013-10-01T10:44:00.000,2013-10-01T10:44:00.000,2013-09-24T00:00:00.000,Transfer,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,Partner
141167,A664237,NaN,2013-10-01T10:44:00.000,2013-10-01T10:44:00.000,2013-09-24T00:00:00.000,Transfer,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,Partner
141168,A664235,NaN,2013-10-01T10:39:00.000,2013-10-01T10:39:00.000,2013-09-24T00:00:00.000,Transfer,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,Partner


In [7]:
df_o, df_i = get_pet_data()

Returning saved csv files.


All the data looks like it's here

<a href='#top'>Back to Top</a>

## Joining the table data <a name='joining'></a>

In [8]:
df_o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141170 entries, 0 to 141169
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   animal_id         141170 non-null  object
 1   name              99536 non-null   object
 2   datetime          141170 non-null  object
 3   monthyear         141170 non-null  object
 4   date_of_birth     141170 non-null  object
 5   outcome_type      141148 non-null  object
 6   animal_type       141170 non-null  object
 7   sex_upon_outcome  141169 non-null  object
 8   age_upon_outcome  141146 non-null  object
 9   breed             141170 non-null  object
 10  color             141170 non-null  object
 11  outcome_subtype   64771 non-null   object
dtypes: object(12)
memory usage: 12.9+ MB


In [9]:
df_i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141303 entries, 0 to 141302
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   animal_id         141303 non-null  object
 1   name              99619 non-null   object
 2   datetime          141303 non-null  object
 3   datetime2         141303 non-null  object
 4   found_location    141303 non-null  object
 5   intake_type       141303 non-null  object
 6   intake_condition  141303 non-null  object
 7   animal_type       141303 non-null  object
 8   sex_upon_intake   141302 non-null  object
 9   age_upon_intake   141303 non-null  object
 10  breed             141303 non-null  object
 11  color             141303 non-null  object
dtypes: object(12)
memory usage: 12.9+ MB


First join the tables .  Start with adding `_i` for intake to all the intake columns to identify

In [10]:
def rename_intake(df):
    return df.add_suffix('_i')

df_i = rename_intake(df_i)

Now join the tables on `animal_id`.  There are repetitions in the `animal_id` columns.  These represent $\approx 20,000$ rows.  Simply drop them and join the two tables with an inner join.

In [11]:
df_i.animal_id_i.value_counts()

A721033    33
A718223    14
A718877    12
A706536    11
A700407     9
           ..
A785847     1
A785845     1
A785952     1
A785955     1
A521520     1
Name: animal_id_i, Length: 126412, dtype: int64

In [19]:
df_o[df_o['animal_id']=='A700407']

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
9760,A700407,Beaux,2021-09-19T15:59:00.000,2021-09-19T15:59:00.000,2014-07-13T00:00:00.000,Adoption,Dog,Neutered Male,7 years,Labrador Retriever Mix,Black,NaN
29139,A700407,Beaux,2019-11-29T18:21:00.000,2019-11-29T18:21:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,5 years,Labrador Retriever Mix,Black,NaN
29353,A700407,Beaux,2019-11-25T16:38:00.000,2019-11-25T16:38:00.000,2014-07-13T00:00:00.000,Rto-Adopt,Dog,Neutered Male,5 years,Labrador Retriever Mix,Black,NaN
45434,A700407,Beaux,2019-02-16T15:41:00.000,2019-02-16T15:41:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black,NaN
62099,A700407,Beaux,2018-02-24T15:43:00.000,2018-02-24T15:43:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,3 years,Labrador Retriever Mix,Black,NaN
80746,A700407,Beaux,2017-01-25T18:09:00.000,2017-01-25T18:09:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,2 years,Labrador Retriever Mix,Black,NaN
97189,A700407,Beaux,2016-02-25T16:45:00.000,2016-02-25T16:45:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,1 year,Labrador Retriever Mix,Black,NaN
110732,A700407,Beaux,2015-06-07T18:16:00.000,2015-06-07T18:16:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,10 months,Labrador Retriever Mix,Black,NaN
113305,A700407,Beaux,2015-04-24T17:53:00.000,2015-04-24T17:53:00.000,2014-07-13T00:00:00.000,Return to Owner,Dog,Neutered Male,9 months,Labrador Retriever Mix,Black,NaN


In [20]:
df_i[df_i['animal_id_i']=='A700407']

,animal_id_i,name_i,datetime_i,datetime2_i,found_location_i,intake_type_i,intake_condition_i,animal_type_i,sex_upon_intake_i,age_upon_intake_i,breed_i,color_i
13782,A700407,Beaux,2021-06-17T11:10:00.000,2021-06-17T11:10:00.000,Cameron And Saint Johns in Austin (TX),Stray,Normal,Dog,Neutered Male,6 years,Labrador Retriever Mix,Black
29069,A700407,Beaux,2019-11-29T16:57:00.000,2019-11-29T16:57:00.000,906 East Leslie Circle in Austin (TX),Stray,Normal,Dog,Neutered Male,5 years,Labrador Retriever Mix,Black
30034,A700407,Beaux,2019-11-09T12:59:00.000,2019-11-09T12:59:00.000,St.Johns And Blessing Avenue in Austin (TX),Stray,Normal,Dog,Neutered Male,5 years,Labrador Retriever Mix,Black
45694,A700407,Beaux,2019-02-16T10:41:00.000,2019-02-16T10:41:00.000,Coronado Hills in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black
62812,A700407,Beaux,2018-02-16T17:35:00.000,2018-02-16T17:35:00.000,5800 Wellington in Austin (TX),Stray,Normal,Dog,Neutered Male,3 years,Labrador Retriever Mix,Black
81143,A700407,Beaux,2017-01-24T18:17:00.000,2017-01-24T18:17:00.000,1111 Rutland Drive in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Labrador Retriever Mix,Black
97697,A700407,Beaux,2016-02-22T13:21:00.000,2016-02-22T13:21:00.000,1807 W Rundberg in Austin (TX),Stray,Normal,Dog,Neutered Male,1 year,Labrador Retriever Mix,Black
111299,A700407,Beaux,2015-05-30T11:01:00.000,2015-05-30T11:01:00.000,Austin (TX),Stray,Normal,Dog,Neutered Male,10 months,Labrador Retriever Mix,Black
114188,A700407,Beaux,2015-04-13T16:20:00.000,2015-04-13T16:20:00.000,6501 Ridge Oak in Austin (TX),Stray,Normal,Dog,Intact Male,9 months,Labrador Retriever Mix,Black


In [ ]:
df_o.animal_id.value_counts()

In [ ]:
def join_tables(df_o, df_i):
    df_i = df_i.drop_duplicates(subset='animal_id_i', keep=False)
    df_o = df_o.drop_duplicates(subset='animal_id', keep=False)
    df = df_o.merge(df_i, how='inner', left_on='animal_id', right_on='animal_id_i')
    df = df.drop(columns=['animal_id_i'])
    return df

df = join_tables(df_o, df_i)

df.info()

<a href='#top'>Back to top</a>
## Function to produce single dataframe <a name='make_dataframe'></a>

In [ ]:
def get_pet_dataframe():
    df_o, df_i = get_pet_data()
    df_i = rename_intake(df_i)
    df = join_tables(df_o, df_i)
    return df

df = get_pet_dataframe()

<a href='#top'>Back to top</a>
## Handling the datetime columns <a name='datetime'></a>

first convert to datetime dtype

In [ ]:
df[['animal_id', 'datetime', 'monthyear', 'datetime_i', 'datetime2_i']]

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['monthyear'] = pd.to_datetime(df['monthyear'])
df['dateime_i'] = pd.to_datetime(df['datetime_i'])
df['datetime2_i'] = pd.to_datetime(df['datetime2_i'])

In [ ]:
df.info()

It doesn't look like there are any mis-matches in the two tables

In [ ]:
df[df['datetime'] != df['monthyear']]

In [ ]:
df[df['datetime_i'] != df['datetime2_i']]

Therefore, make two columns: `outcome_date` and `intake_date` that contains the day of each

In [ ]:
def make_date_columns(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['monthyear'] = pd.to_datetime(df['monthyear'])
    df['datetime_i'] = pd.to_datetime(df['datetime_i'])
    df['datetime2_i'] = pd.to_datetime(df['datetime2_i'])
    df['outcome_date'] = df['monthyear'].dt.strftime('%m %d, %Y')
    df['intake_date'] = df['monthyear'].dt.strftime('%m %d, %Y')
    df = df.drop(columns = ['datetime', 'monthyear', 'datetime_i', 'datetime2_i'])
    return df

make_date_columns(get_pet_dataframe()).info()

<a href='#top'>Back to top</a>

## Checking data integrity of select columns <a name='integrity'></a>

all the animal types are the same

In [ ]:
df[df['animal_type'] != df['animal_type_i']]

Mis-matched names are `NaN`.

In [ ]:
df[df['name'] != df['name_i']][['name', 'name_i']].value_counts(dropna=False)

All the breeds are consistent

In [ ]:
df[df['breed'] != df['breed_i']]

color is consistent

In [ ]:
df[df['color'] != df['color_i']]

<a href='#top'>Back to top</a>

## Considering nulls <a name='nulls'></a>

name column can be filled with string:

In [ ]:
df[df['name'] != df['name_i']][['name', 'name_i']].value_counts(dropna=False)

In [ ]:
df.info()

There are only fifteen `NaN`.  These can be dropped safely

In [ ]:
df.outcome_type.value_counts(dropna=False)

Replace outcome_subtype NaN's with no subtype. Consider the cross tab for 

In [ ]:
df.outcome_subtype.value_counts(dropna=False)

In [ ]:
pd.crosstab(df.outcome_subtype, df.outcome_type)

SCRP is the Stray Cat Return Program.  A spay and release program for cats.

In [ ]:
df[df.outcome_subtype == 'SCRP'].animal_type.value_counts()

Drop the one NaN

In [ ]:
df.sex_upon_outcome.value_counts(dropna=False)

In [ ]:
df.age_upon_outcome.value_counts(dropna=False)

In [ ]:
df.sex_upon_intake_i.value_counts(dropna=False)

## Function to fill nulls and drop nulls

In [ ]:
def null_fill_and_drop(df):
    df.name = df.name.fillna('no name')
    df.outcome_subtype = df.outcome_subtype.fillna('no subtype')
    df = df.drop(columns=['name_i', 'breed_i', 'color_i', 'animal_type_i'])
    df = df.dropna()
    return df

df = null_fill_and_drop(get_pet_dataframe())

In [ ]:
df.info()

<a href='#top'>Back to top</a>

## Fixing the age columns <a name='age'></a>

The age column contains a variety of strings that can be turned into a number.  The smallest unit present is days.  Therefore convert all into days. 



In [ ]:
df[['age_upon_outcome']].value_counts()[:5]

In [ ]:
def convert_age_column(df):
    new_data = []
    multipliers = {
        'day': 1,
        'days': 1,
        'week':7,
        'weeks':7,
        'month': 30.5,
        'months':30.5,
        'year':365.25,
        'years':365.25
    }
    for i, row in df.iterrows():
        outcome_age_split = row['age_upon_outcome'].split()
        outcome_age_calc = int(outcome_age_split[0])*multipliers[outcome_age_split[1]]
        intake_age_split = row['age_upon_intake_i'].split()
        intake_age_calc = int(intake_age_split[0])*multipliers[intake_age_split[1]]
        datum_calc = {
            'animal_id':row['animal_id'],
            'age_at_outcome':outcome_age_calc,
            'age_at_intake':intake_age_calc
        }
        new_data.append(datum_calc)
    df_calc = pd.DataFrame(new_data)
    df = df.merge(df_calc)
    return df.drop(columns = ['age_upon_outcome', 'age_upon_intake_i'])

df = convert_age_column(df)

In [ ]:
df.info()

<a href='#top'>Back to Top</a>

## Trying out the scripts from `.py` files <a name='scripts'></a>


In [ ]:
import wrangle

In [ ]:
df = wrangle.get_pet_dataframe()
df

In [ ]:
df = wrangle.prepare_pet_dataframe(df)
df

In [ ]:
df.info()

In [ ]:
df.rename(columns = {'found_location_i':'found_location',
                    'intake_type_i':'intake_type',
                    'intake_condition_i': 'intake_condition',
                    'sex_upon_intake_i': 'sex_upon_intake'})